TMDB Movie Data Analysis Project - Phase 3: KPI Analysis & Rankings
Step 4: Complete Analysis, Rankings, and Insights

This script performs all required KPI calculations and analysis.


In [11]:
# ============================================================================
# IMPORTS
# ============================================================================
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:

# ============================================================================
# LOAD CLEANED DATA
# ============================================================================

# Load the cleaned data from Phase 2
df = pd.read_csv('movies_clean.csv')

print(f"\n Loaded cleaned data: {df.shape[0]} movies × {df.shape[1]} columns")
print(f" Date range: {pd.to_datetime(df['release_date']).min().year} - {pd.to_datetime(df['release_date']).max().year}")

PHASE 3: KPI ANALYSIS & RANKINGS

 Loaded cleaned data: 18 movies × 22 columns
 Date range: 1997 - 2019


In [13]:
# ============================================================================
# STEP 1: Calculate KPIs (Profit & ROI)
# ============================================================================


# 1.1 Calculate Profit (in millions USD)
df['profit_musd'] = df['revenue_musd'] - df['budget_musd']

# 1.2 Calculate ROI (Return on Investment)
# ROI = (Revenue / Budget) × 100 to get percentage
# Only calculate where budget is not zero or NaN
df['roi_percent'] = np.where(
    (df['budget_musd'].notna()) & (df['budget_musd'] > 0),
    (df['revenue_musd'] / df['budget_musd']) * 100,
    np.nan
)

# Show summary statistics
print(f"\n Profit calculated for {df['profit_musd'].notna().sum()} movies")
print(f" ROI calculated for {df['roi_percent'].notna().sum()} movies")

print("\nProfit Statistics (in millions USD):")
print(f"  • Mean: ${df['profit_musd'].mean():.2f}M")
print(f"  • Median: ${df['profit_musd'].median():.2f}M")
print(f"  • Max: ${df['profit_musd'].max():.2f}M")
print(f"  • Min: ${df['profit_musd'].min():.2f}M")

print("\nROI Statistics (percentage):")
print(f"  • Mean: {df['roi_percent'].mean():.2f}%")
print(f"  • Median: {df['roi_percent'].median():.2f}%")
print(f"  • Max: {df['roi_percent'].max():.2f}%")
print(f"  • Min: {df['roi_percent'].min():.2f}%")


 Profit calculated for 18 movies
 ROI calculated for 18 movies

Profit Statistics (in millions USD):
  • Mean: $1478.05M
  • Median: $1301.25M
  • Max: $2686.71M
  • Min: $1032.70M

ROI Statistics (percentage):
  • Mean: 817.49%
  • Median: 788.84%
  • Max: 1233.63%
  • Min: 444.23%


In [14]:
# ============================================================================
# STEP 2: Create User-Defined Function for Rankings
# ============================================================================

def rank_movies(dataframe, sort_column, ascending=False, top_n=10, 
                filter_condition=None, display_columns=None):
    """
    Universal function to rank movies based on any metric.
    
    Parameters:
    -----------
    dataframe : pd.DataFrame
        The movies dataframe
    sort_column : str
        Column name to sort by
    ascending : bool
        True for lowest first, False for highest first
    top_n : int
        Number of top movies to return
    filter_condition : pd.Series or None
        Boolean mask to filter dataframe before ranking
    display_columns : list or None
        Columns to display in results
    
    Returns:
    --------
    pd.DataFrame
        Top N ranked movies
    """
    
    # Apply filter if provided
    if filter_condition is not None:
        df_filtered = dataframe[filter_condition].copy()
    else:
        df_filtered = dataframe.copy()
    
    # Remove rows where sort column is NaN
    df_filtered = df_filtered[df_filtered[sort_column].notna()]
    
    # Sort
    df_ranked = df_filtered.sort_values(by=sort_column, ascending=ascending)
    
    # Get top N
    df_top = df_ranked.head(top_n)
    
    # Select display columns if specified
    if display_columns:
        df_top = df_top[display_columns]
    
    return df_top

In [15]:

# ============================================================================
# STEP 3: Generate All 10 Required Rankings
# ============================================================================

# Define columns to display for rankings
ranking_display_cols = ['title', 'release_date', 'budget_musd', 'revenue_musd', 
                        'profit_musd', 'roi_percent', 'vote_average', 'vote_count', 'popularity']

# 2.1 Highest Revenue
print("\n" + "-"*70)
print("2.1 TOP 10 MOVIES BY HIGHEST REVENUE")
print("-"*70)
top_revenue = rank_movies(
    df, 
    sort_column='revenue_musd',
    ascending=False,
    top_n=10,
    display_columns=['title', 'revenue_musd', 'budget_musd', 'release_date']
)
print(top_revenue.to_string(index=False))

# 2.2 Highest Budget
print("\n" + "-"*70)
print("2.2 TOP 10 MOVIES BY HIGHEST BUDGET")
print("-"*70)
top_budget = rank_movies(
    df,
    sort_column='budget_musd',
    ascending=False,
    top_n=10,
    display_columns=['title', 'budget_musd', 'revenue_musd', 'release_date']
)
print(top_budget.to_string(index=False))

# 2.3 Highest Profit
print("\n" + "-"*70)
print("2.3 TOP 10 MOVIES BY HIGHEST PROFIT")
print("-"*70)
top_profit = rank_movies(
    df,
    sort_column='profit_musd',
    ascending=False,
    top_n=10,
    display_columns=['title', 'profit_musd', 'revenue_musd', 'budget_musd', 'release_date']
)
print(top_profit.to_string(index=False))

# 2.4 Lowest Profit (Biggest Flops)
print("\n" + "-"*70)
print("2.4 TOP 10 MOVIES BY LOWEST PROFIT (Biggest Flops)")
print("-"*70)
lowest_profit = rank_movies(
    df,
    sort_column='profit_musd',
    ascending=True,
    top_n=10,
    display_columns=['title', 'profit_musd', 'revenue_musd', 'budget_musd', 'release_date']
)
print(lowest_profit.to_string(index=False))

# 2.5 Highest ROI (Budget ≥ 10M only)
print("\n" + "-"*70)
print("2.5 TOP 10 MOVIES BY HIGHEST ROI (Budget ≥ $10M)")
print("-"*70)
top_roi = rank_movies(
    df,
    sort_column='roi_percent',
    ascending=False,
    top_n=10,
    filter_condition=(df['budget_musd'] >= 10),
    display_columns=['title', 'roi_percent', 'budget_musd', 'revenue_musd', 'release_date']
)
print(top_roi.to_string(index=False))

# 2.6 Lowest ROI (Budget ≥ 10M only)
print("\n" + "-"*70)
print("2.6 TOP 10 MOVIES BY LOWEST ROI (Budget ≥ $10M)")
print("-"*70)
lowest_roi = rank_movies(
    df,
    sort_column='roi_percent',
    ascending=True,
    top_n=10,
    filter_condition=(df['budget_musd'] >= 10),
    display_columns=['title', 'roi_percent', 'budget_musd', 'revenue_musd', 'release_date']
)
print(lowest_roi.to_string(index=False))

# 2.7 Most Voted Movies
print("\n" + "-"*70)
print("2.7 TOP 10 MOST VOTED MOVIES")
print("-"*70)
most_voted = rank_movies(
    df,
    sort_column='vote_count',
    ascending=False,
    top_n=10,
    display_columns=['title', 'vote_count', 'vote_average', 'release_date']
)
print(most_voted.to_string(index=False))

# 2.8 Highest Rated (≥10 votes)
print("\n" + "-"*70)
print("2.8 TOP 10 HIGHEST RATED MOVIES (with ≥10 votes)")
print("-"*70)
highest_rated = rank_movies(
    df,
    sort_column='vote_average',
    ascending=False,
    top_n=10,
    filter_condition=(df['vote_count'] >= 10),
    display_columns=['title', 'vote_average', 'vote_count', 'release_date']
)
print(highest_rated.to_string(index=False))

# 2.9 Lowest Rated (≥10 votes)
print("\n" + "-"*70)
print("2.9 TOP 10 LOWEST RATED MOVIES (with ≥10 votes)")
print("-"*70)
lowest_rated = rank_movies(
    df,
    sort_column='vote_average',
    ascending=True,
    top_n=10,
    filter_condition=(df['vote_count'] >= 10),
    display_columns=['title', 'vote_average', 'vote_count', 'release_date']
)
print(lowest_rated.to_string(index=False))

# 2.10 Most Popular Movies
print("\n" + "-"*70)
print("2.10 TOP 10 MOST POPULAR MOVIES")
print("-"*70)
most_popular = rank_movies(
    df,
    sort_column='popularity',
    ascending=False,
    top_n=10,
    display_columns=['title', 'popularity', 'vote_average', 'release_date']
)
print(most_popular.to_string(index=False))


----------------------------------------------------------------------
2.1 TOP 10 MOVIES BY HIGHEST REVENUE
----------------------------------------------------------------------
                       title  revenue_musd  budget_musd release_date
                      Avatar   2923.706026        237.0   2009-12-15
           Avengers: Endgame   2799.439100        356.0   2019-04-24
                     Titanic   2264.162353        200.0   1997-11-18
Star Wars: The Force Awakens   2068.223624        245.0   2015-12-15
      Avengers: Infinity War   2052.415039        300.0   2018-04-25
              Jurassic World   1671.537444        150.0   2015-06-06
               The Lion King   1662.020819        260.0   2019-07-12
                The Avengers   1518.815515        220.0   2012-04-25
                   Furious 7   1515.400000        190.0   2015-04-01
                   Frozen II   1453.683476        150.0   2019-11-20

------------------------------------------------------------

In [16]:
# ============================================================================
# STEP 4: Advanced Movie Filtering & Search Queries
# ============================================================================

# 3.1 Search 1: Best-rated Sci-Fi Action movies starring Bruce Willis
print("\n" + "-"*70)
print("3.1 SEARCH: Best Sci-Fi Action movies with Bruce Willis")
print("-"*70)

# Create filters - Convert to string first to avoid AttributeError
has_scifi = df['genres'].notna() & df['genres'].astype(str).str.contains('Science Fiction', case=False, na=False)
has_action = df['genres'].notna() & df['genres'].astype(str).str.contains('Action', case=False, na=False)
has_bruce = df['cast'].notna() & df['cast'].astype(str).str.contains('Bruce Willis', case=False, na=False)

search1_results = df[has_scifi & has_action & has_bruce].sort_values(
    by='vote_average', 
    ascending=False
)

if len(search1_results) > 0:
    print(search1_results[['title', 'genres', 'cast', 'vote_average', 'release_date']].to_string(index=False))
else:
    print("No movies found matching criteria (Sci-Fi Action with Bruce Willis)")

# 3.2 Search 2: Uma Thurman directed by Quentin Tarantino
print("\n" + "-"*70)
print("3.2 SEARCH: Uma Thurman movies directed by Quentin Tarantino")
print("-"*70)

# Convert to string first
has_uma = df['cast'].notna() & df['cast'].astype(str).str.contains('Uma Thurman', case=False, na=False)
has_tarantino = df['director'].notna() & df['director'].astype(str).str.contains('Quentin Tarantino', case=False, na=False)

search2_results = df[has_uma & has_tarantino].sort_values(
    by='runtime',
    ascending=True
)

if len(search2_results) > 0:
    print(search2_results[['title', 'director', 'cast', 'runtime', 'release_date']].to_string(index=False))
else:
    print("No movies found matching criteria (Uma Thurman + Quentin Tarantino)")


----------------------------------------------------------------------
3.1 SEARCH: Best Sci-Fi Action movies with Bruce Willis
----------------------------------------------------------------------
No movies found matching criteria (Sci-Fi Action with Bruce Willis)

----------------------------------------------------------------------
3.2 SEARCH: Uma Thurman movies directed by Quentin Tarantino
----------------------------------------------------------------------
No movies found matching criteria (Uma Thurman + Quentin Tarantino)


In [17]:
# ============================================================================
# STEP 5: Franchise vs Standalone Movie Performance
# ============================================================================

# Separate franchise and standalone movies
df['is_franchise'] = df['belongs_to_collection'].notna()

franchise_movies = df[df['is_franchise'] == True]
standalone_movies = df[df['is_franchise'] == False]

print(f"\nFranchise movies: {len(franchise_movies)}")
print(f"Standalone movies: {len(standalone_movies)}")

# Create comparison dataframe
comparison_metrics = {
    'Metric': [
        'Count',
        'Mean Revenue (M USD)',
        'Median Revenue (M USD)',
        'Mean Budget (M USD)',
        'Median Budget (M USD)',
        'Mean ROI (%)',
        'Median ROI (%)',
        'Mean Profit (M USD)',
        'Mean Popularity',
        'Mean Rating'
    ],
    'Franchise': [
        len(franchise_movies),
        franchise_movies['revenue_musd'].mean(),
        franchise_movies['revenue_musd'].median(),
        franchise_movies['budget_musd'].mean(),
        franchise_movies['budget_musd'].median(),
        franchise_movies['roi_percent'].mean(),
        franchise_movies['roi_percent'].median(),
        franchise_movies['profit_musd'].mean(),
        franchise_movies['popularity'].mean(),
        franchise_movies['vote_average'].mean()
    ],
    'Standalone': [
        len(standalone_movies),
        standalone_movies['revenue_musd'].mean(),
        standalone_movies['revenue_musd'].median(),
        standalone_movies['budget_musd'].mean(),
        standalone_movies['budget_musd'].median(),
        standalone_movies['roi_percent'].mean(),
        standalone_movies['roi_percent'].median(),
        standalone_movies['profit_musd'].mean(),
        standalone_movies['popularity'].mean(),
        standalone_movies['vote_average'].mean()
    ]
}

comparison_df = pd.DataFrame(comparison_metrics)

print("\n" + "-"*70)
print("FRANCHISE VS STANDALONE PERFORMANCE")
print("-"*70)
print(comparison_df.to_string(index=False))


Franchise movies: 16
Standalone movies: 2

----------------------------------------------------------------------
FRANCHISE VS STANDALONE PERFORMANCE
----------------------------------------------------------------------
                Metric   Franchise  Standalone
                 Count   16.000000    2.000000
  Mean Revenue (M USD) 1682.668411 1765.139159
Median Revenue (M USD) 1484.541738 1765.139159
   Mean Budget (M USD)  218.000000  180.000000
 Median Budget (M USD)  210.000000  180.000000
          Mean ROI (%)  799.461013  961.701827
        Median ROI (%)  778.611719  961.701827
   Mean Profit (M USD) 1464.668411 1585.139159
       Mean Popularity   17.026169   19.218700
           Mean Rating    7.382625    7.450000


In [18]:
# ============================================================================
# STEP 6: Most Successful Movie Franchises
# ============================================================================

# Group by franchise
franchise_analysis = franchise_movies.groupby('belongs_to_collection').agg({
    'id': 'count',  # Number of movies
    'budget_musd': ['sum', 'mean'],
    'revenue_musd': ['sum', 'mean'],
    'vote_average': 'mean',
    'profit_musd': 'sum'
}).round(2)

# Flatten column names
franchise_analysis.columns = ['_'.join(col) for col in franchise_analysis.columns]
franchise_analysis = franchise_analysis.rename(columns={
    'id_count': 'movie_count',
    'budget_musd_sum': 'total_budget_musd',
    'budget_musd_mean': 'mean_budget_musd',
    'revenue_musd_sum': 'total_revenue_musd',
    'revenue_musd_mean': 'mean_revenue_musd',
    'vote_average_mean': 'mean_rating',
    'profit_musd_sum': 'total_profit_musd'
})

# Sort by total revenue
franchise_analysis = franchise_analysis.sort_values('total_revenue_musd', ascending=False)

print("\n" + "-"*70)
print("TOP FRANCHISES BY TOTAL REVENUE")
print("-"*70)
print(franchise_analysis.head(10).to_string())


----------------------------------------------------------------------
TOP FRANCHISES BY TOTAL REVENUE
----------------------------------------------------------------------
                                     movie_count  total_budget_musd  mean_budget_musd  total_revenue_musd  mean_revenue_musd  mean_rating  total_profit_musd
belongs_to_collection                                                                                                                                       
The Avengers Collection                        4             1111.0            277.75             7776.07            1944.02         7.90            6665.07
Star Wars Collection                           2              545.0            272.50             3400.92            1700.46         7.01            2855.92
Jurassic Park Collection                       2              320.0            160.00             2982.01            1491.00         6.62            2662.01
Avatar Collection                       

In [19]:
# ============================================================================
# STEP 7: Most Successful Directors
# ============================================================================

# Filter movies with director info
df_with_director = df[df['director'].notna()].copy()

# Group by director
director_analysis = df_with_director.groupby('director').agg({
    'id': 'count',  # Number of movies
    'revenue_musd': 'sum',
    'vote_average': 'mean',
    'profit_musd': 'sum'
}).round(2)

director_analysis.columns = ['movie_count', 'total_revenue_musd', 'mean_rating', 'total_profit_musd']

# Sort by total revenue
director_analysis = director_analysis.sort_values('total_revenue_musd', ascending=False)

print("\n" + "-"*70)
print("TOP DIRECTORS BY TOTAL REVENUE")
print("-"*70)
print(director_analysis.head(10).to_string())


----------------------------------------------------------------------
TOP DIRECTORS BY TOTAL REVENUE
----------------------------------------------------------------------
                 movie_count  total_revenue_musd  mean_rating  total_profit_musd
director                                                                        
James Cameron              2             5187.87         7.75            4750.87
Joss Whedon                2             2924.22         7.57            2469.22
Anthony Russo              1             2799.44         8.24            2443.44
J.J. Abrams                1             2068.22         7.26            1823.22
Joe Russo                  1             2052.42         8.24            1752.42
Colin Trevorrow            1             1671.54         6.70            1521.54
Jon Favreau                1             1662.02         7.10            1402.02
James Wan                  1             1515.40         7.20            1325.40
Jennifer Lee    

In [20]:
# ============================================================================
# SAVE ANALYSIS RESULTS
# ============================================================================

# Saving updated dataframe with KPIs
df.to_csv('movies_with_kpis.csv', index=False)
print(" Saved movies with KPIs to: movies_with_kpis.csv")

# Save all rankings to Excel (if openpyxl is available)
try:
    with pd.ExcelWriter('movie_rankings.xlsx', engine='openpyxl') as writer:
        top_revenue.to_excel(writer, sheet_name='Top Revenue', index=False)
        top_budget.to_excel(writer, sheet_name='Top Budget', index=False)
        top_profit.to_excel(writer, sheet_name='Top Profit', index=False)
        lowest_profit.to_excel(writer, sheet_name='Lowest Profit', index=False)
        top_roi.to_excel(writer, sheet_name='Top ROI', index=False)
        lowest_roi.to_excel(writer, sheet_name='Lowest ROI', index=False)
        most_voted.to_excel(writer, sheet_name='Most Voted', index=False)
        highest_rated.to_excel(writer, sheet_name='Highest Rated', index=False)
        lowest_rated.to_excel(writer, sheet_name='Lowest Rated', index=False)
        most_popular.to_excel(writer, sheet_name='Most Popular', index=False)
        comparison_df.to_excel(writer, sheet_name='Franchise vs Standalone', index=False)
        franchise_analysis.to_excel(writer, sheet_name='Top Franchises')
        director_analysis.to_excel(writer, sheet_name='Top Directors')
    
    print(" Saved all rankings to: movie_rankings.xlsx")
except ImportError:
    print(" Excel export skipped (openpyxl not installed)")
    print("  Install with: pip install openpyxl")


 Saved movies with KPIs to: movies_with_kpis.csv
 Saved all rankings to: movie_rankings.xlsx
